In [20]:
%load_ext nvcc4jupyter

from nvcc4jupyter import set_defaults
set_defaults(compiler_args='-arch=sm_100a -Xptxas=-v -O0')

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


# GOD I AM SO HAPPY, SO HAPPY, THAT JUPYTER IN REMOTE SSH IS WORKING AGAIN! LETS DO SOME SMEM STUFF BABYYYYYY. 

In [3]:
%%cuda

#include<stdio.h> 
#include<stdlib.h>
#include<cuda.h>
#include<cuda_runtime.h> 

constexpr int N_iter = 100000;
constexpr int N_warps = 32;
constexpr int N_lanes = 32;
constexpr int N_blocks = 1;

__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}

__global__ void smem_no_bc()
{
  __shared__ float A[N_warps][N_lanes];
  int warp_id = threadIdx.y; 
  int lane_id = threadIdx.x; 
  
  float* ptr = &A[warp_id][lane_id];
  int addr = (int)ptr & 0xFFFF; //#0x is just a prefix meaning its hexadecimal 
  //# each F is a 15, and each digit is worth that power of 16. 
  //# in fact this number is 2^16 + 1, which means we are bit masking with the and 
  //# and extracting the first 16 digits of our pointer. but why? 
  /*
  """
  well really, each address is worth a byte (8 bits) and so how many bytes do we have in this shared memory?
  we have 8*32 bytes, so we have 8*8*32 bits, so its enough for our adress to be that big 
  well 8*8*32 = 2^6 * 2^5 = 2^11, 
  
  but the adress itself, if you do addr + 1, you are moving 8 bits (or 1 byte) 
  so then, if we have 2^16 - 1 addrs, we are guuci, but idk whats more to this nvm
  """
  */
  float r1;
  int n_clocks = 0;
  unsigned long long start;
  unsigned long long end;
  
  
  for (int j = 0; j < N_iter; j++) {
      
      start = get_clock64();
      
      asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                      : "=f"(r1)
                      : "r"(addr));
      
      end = get_clock64();
      
      if (threadIdx.x == 0 && threadIdx.y == 0)
        {
          n_clocks += end - start;
        }
  }
  
  __syncthreads(); 
  if (threadIdx.x == 0 && threadIdx.y == 0)
  {
    printf("%d \n", n_clocks);
  }
  
}

int main()
{
   
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

 
  cudaEventRecord(start, 0);

 
  //#weirdly enough, it is actually threadidx.x that has stride one, even tho its 
  //#on the left of the tuple. so we gotta be careful about that launch
  dim3 threads(N_lanes, N_warps);
  dim3 grid(N_blocks);
  
  smem_no_bc<<<grid, threads>>>();


  cudaEventRecord(stop, 0);
  cudaEventSynchronize(stop);

  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("GPU Kernel time: %f ms\n", milliseconds);
  
  
  /*
  """
  so how do we calculate our memory transfer speed, well, we have N_warps warps, 
  so per iteration, we issue N_warp loads per bank, so our speed will be 
  N_loads/bank/second. 
  so for N_iter iterations, 
  smem_bank_speed = (N_warps*N_iter)/time 
  """
  */
  float seconds = milliseconds*0.001;
  float mega = 1000000.0;
  printf("mega_loads/bank/second = %f \n", (N_warps*N_iter)/(seconds*mega));
  
  int deviceId;
  cudaGetDevice(&deviceId);
  
  cudaDeviceProp prop;
  cudaGetDeviceProperties(&prop, deviceId);
  

  double smClockMHz = prop.clockRate / 1000.0;
  
  printf("Device: %s\n", prop.name);
  printf("Base SM Clock Rate: %.2f MHz\n", smClockMHz);
  printf("latency? %f", (2711953.0)/(N_iter));
}

2711953 
GPU Kernel time: 25.060768 ms
mega_loads/bank/second = 127.689621 
Device: NVIDIA GeForce RTX 5090
Base SM Clock Rate: 2437.00 MHz
latency? 27.119530


In [4]:
"""
one wonders why I am not getting 1 load per bank per clock cycle. 

"""

'\none wonders why I am not getting 1 load per bank per clock cycle. \n\n'

In [5]:
%%cuda
#include <stdio.h> 
#include <stdlib.h>
#include <cuda_runtime.h> 
#include <device_launch_parameters.h>

// Define a safe macro for CUDA error checking
#define CUDA_CHECK(call)                                                          \
    do {                                                                          \
        cudaError_t err = call;                                                   \
        if (err != cudaSuccess) {                                                 \
            fprintf(stderr, "CUDA error at %s:%d: %s\n",                          \
                    __FILE__, __LINE__, cudaGetErrorString(err));                 \
            exit(EXIT_FAILURE);                                                   \
        }                                                                         \
    } while (0)

constexpr int N_iter = 2000;
constexpr int N_threads = 1;
constexpr int N_blocks = 1;

// 64-bit clock counter: This is correct for PTX `clock64`.
__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}

// NOTE: Kernel signature requires 4 arguments.
__global__ void scalar_smem_latency (float* In, float* Out, unsigned long long *starts, unsigned long long *stops)
{
  int t = threadIdx.x; 
  __shared__ float S[N_iter]; 
  float r1; 
 
  unsigned long long s; 
  unsigned long long e; 
  
  for (int j = 0; j < N_iter; j++)

  { 
      if(j < 10)
      {
        printf("%f  ", In[t+j]);
      }

      S[t+j] = In[t+j];
  }
  
  __syncthreads();
  
  for (int j = 0; j < N_iter; j+=8)
  {
    float* ptr_0 = &S[t + j]; 
    int addr_0 = (int)ptr_0 & 0xFFFF;
    float* ptr_1 = &S[t + j + 1]; 
    int addr_1 = (int)ptr_1 & 0xFFFF;
    float* ptr_2 = &S[t + j + 2]; 
    int addr_2 = (int)ptr_2 & 0xFFFF;
    float* ptr_3 = &S[t + j + 3]; 
    int addr_3 = (int)ptr_3 & 0xFFFF;
    float* ptr_4 = &S[t + j + 4]; 
    int addr_4 = (int)ptr_4 & 0xFFFF;
    float* ptr_5 = &S[t + j + 5]; 
    int addr_5 = (int)ptr_5 & 0xFFFF;
    float* ptr_6 = &S[t + j + 6]; 
    int addr_6 = (int)ptr_6 & 0xFFFF;
    float* ptr_7 = &S[t + j + 7]; 
    int addr_7 = (int)ptr_7 & 0xFFFF;
    Out[t + j] = r1;
    s = get_clock64();
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_0));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_1));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                      : "=f"(r1)
                      : "r"(addr_2));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_3));
        asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_4));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_6));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                      : "=f"(r1)
                      : "r"(addr_6));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_7));
      
    e = get_clock64();
    starts[t + j] = s; 
    stops[t + j] = e;
    Out[t + j + 7] = r1;

  }
  
}


int main()
{
    //# --- 1. Variable Declarations and Size Calculations ---
    unsigned long long *h_start, *h_stop;
    unsigned long long *d_start, *d_stop;
    
    //# Size for cycle arrays (N_iter measurements for 1 thread)
    size_t size_cycles = N_iter * sizeof(unsigned long long);
    
    //# Size for I/O float arrays
    size_t size_IO = N_iter * sizeof(float);
    float* h_In, *h_out;
    float* D_In, *D_out; // Device pointers for In and Out (required by kernel signature)

    //# --- 2. Host and Device Memory Allocation and Initialization ---
    
    //# Allocate Pinned Host memory for In and Out (for setup)
    CUDA_CHECK(cudaHostAlloc(&h_In, size_IO, cudaHostAllocDefault));
    CUDA_CHECK(cudaHostAlloc(&h_out, size_IO, cudaHostAllocDefault)); 
    
    //# Corrected loop syntax: semicolon instead of comma in the condition
    for (int i = 0; i < N_iter; i++) 
    {
        h_In[i] = (float) (i+1) / (100.0); 
        h_out[i] = 0.0f; // #Use 0.0f for float literal
    }
    
    //# Allocate Pinned Host memory for start/stop cycles
    CUDA_CHECK(cudaHostAlloc(&h_start, size_cycles, cudaHostAllocDefault));
    CUDA_CHECK(cudaHostAlloc(&h_stop, size_cycles, cudaHostAllocDefault));
    
    // Allocate Device memory for all 4 kernel arguments
    CUDA_CHECK(cudaMalloc(&d_start, size_cycles));
    CUDA_CHECK(cudaMalloc(&d_stop, size_cycles));
    CUDA_CHECK(cudaMalloc(&D_In, size_IO));
    CUDA_CHECK(cudaMalloc(&D_out, size_IO));

    // Copy input data to the device (D_In)
    CUDA_CHECK(cudaMemcpy(D_In, h_In, size_IO, cudaMemcpyHostToDevice));
    
    // --- 3. Kernel Launch ---

    printf("Launching kernel with N_blocks=%d and N_threads=%d...\n", N_blocks, N_threads);

    // Launch the kernel with all 4 required arguments
    scalar_smem_latency<<<N_blocks, N_threads>>>(D_In, D_out, d_start, d_stop);
    
    // Wait for the kernel to finish
    CUDA_CHECK(cudaDeviceSynchronize());
    
    // --- 4. Data Transfer and Processing ---
    
    // Copy the cycle counters back to the host
    CUDA_CHECK(cudaMemcpy(h_start, d_start, size_cycles, cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_stop, d_stop, size_cycles, cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_out, D_out, size_IO, cudaMemcpyDeviceToHost));

    float avg = 0.0;
    
    for (int i = 0; i < N_iter; i+=8)
    {
        // Safe subtraction using unsigned long long
        unsigned long long diff = h_stop[i] - h_start[i];
        printf("value of r1 at iteration %d: %f \n", i, h_out[i]);
        printf("Latency at iteration %d: %llu cycles \n", i, diff);
        avg += ((float)diff/8.0);

    }
    printf("average latency %f \n", avg/250.0);
    // --- 5. Cleanup ---
    
    cudaFree(d_start);
    cudaFree(d_stop);
    cudaFree(D_In);
    cudaFree(D_out);
    
    cudaFreeHost(h_start);
    cudaFreeHost(h_stop);
    cudaFreeHost(h_In);
    cudaFreeHost(h_out);
    
    return 0;
}

Launching kernel with N_blocks=1 and N_threads=1...
0.010000  0.020000  0.030000  0.040000  0.050000  0.060000  0.070000  0.080000  0.090000  0.100000  value of r1 at iteration 0: 0.000000 
Latency at iteration 0: 45 cycles 
value of r1 at iteration 8: 0.080000 
Latency at iteration 8: 53 cycles 
value of r1 at iteration 16: 0.160000 
Latency at iteration 16: 45 cycles 
value of r1 at iteration 24: 0.240000 
Latency at iteration 24: 53 cycles 
value of r1 at iteration 32: 0.320000 
Latency at iteration 32: 45 cycles 
value of r1 at iteration 40: 0.400000 
Latency at iteration 40: 53 cycles 
value of r1 at iteration 48: 0.480000 
Latency at iteration 48: 45 cycles 
value of r1 at iteration 56: 0.560000 
Latency at iteration 56: 53 cycles 
value of r1 at iteration 64: 0.640000 
Latency at iteration 64: 45 cycles 
value of r1 at iteration 72: 0.720000 
Latency at iteration 72: 53 cycles 
value of r1 at iteration 80: 0.800000 
Latency at iteration 80: 45 cycles 
value of r1 at iteration 88

The problem is, that scheluling and pipelining is heavily optimized by the GPU, so one HAS TO make the next addrs of load dependent of the previous, we will always compute an N chased pointer sequence and do many such iterations of that sequence. 

The problem is, even if we are writing to the same register each time, if that register is not being used, each load can be pipelined such that only the final register being used or stored needs to finish, the rest of the load issuing can overlap within the latency of previous loads 

if we do be like 

r1 = load(addr) 
r1 = v + r1 

in a loop, this MIGHT still be pipelined, cause we kow the addresses before hand. 

The best way is to make the next addr dependent on the loaded value of the register, hence pointer chasing

In [6]:
%%cuda
#include <stdio.h> 
#include <stdlib.h>
#include <cuda_runtime.h> 
#include <device_launch_parameters.h>


#define CUDA_CHECK(call)                                                          \
    do {                                                                          \
        cudaError_t err = call;                                                   \
        if (err != cudaSuccess) {                                                 \
            fprintf(stderr, "CUDA error at %s:%d: %s\n",                          \
                    __FILE__, __LINE__, cudaGetErrorString(err));                 \
            exit(EXIT_FAILURE);                                                   \
        }                                                                         \
    } while (0)

constexpr int N_iter = 2000;
constexpr int N_chase = 64; 
constexpr int chase_iter = 200;

// 64-bit clock counter: This is correct for PTX `clock64`.
__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}
//#we will always start the pointer chase at index 0
__global__ void smem_chase (float* chase, unsigned long long * clock)
{

    int t = threadIdx.x; 
    __shared__ float S[N_chase];
    for (int j = 0; j < N_chase; j++)
    {
        S[t+j] = chase[t+j];
    }
    
    for (int k = 0; k < N_iter; k++)
    {
        int idx = 0; 
        int addr = (int)(&S[t+idx]); 
        unsigned long long s,e; 
        float r1; 
        
        s = get_clock64();
        for (int q = 0; q < N_chase; q++)
        {
            asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr));
            idx = (int) r1;
            addr = (int)(&S[t+idx]);
        }
        e = get_clock64();
        
        clock[t+k] = e-s;
        
    }
    
}
int main()
{
    size_t arr_size = N_chase * sizeof(float);
    size_t clock_arr_size = N_iter * sizeof(unsigned long long);
    
    float* h_chase, *d_chase;
    unsigned long long * h_clock, *d_clock;
    
    CUDA_CHECK(cudaHostAlloc(&h_chase, arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaHostAlloc(&h_clock, clock_arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaMalloc(&d_chase, arr_size));
    CUDA_CHECK(cudaMalloc(&d_clock, clock_arr_size));
    
    int index = 0;
    
    for (int q = 0; q < chase_iter - 1; q++)
    {
        h_chase[index] = (float) (rand() % N_chase); 
        index = (int) h_chase[index];
        //#printf("iteration : %d : index : %d \n", q,index);
    }
    h_chase[index] = 0.0;
    
    CUDA_CHECK(cudaMemcpy(d_chase, h_chase, arr_size, cudaMemcpyHostToDevice));
    smem_chase<<<1,1>>>(d_chase, d_clock); 
    CUDA_CHECK(cudaMemcpy(h_clock, d_clock, clock_arr_size, cudaMemcpyDeviceToHost));
    
    for (int i = 0; i < N_iter; i++)
    {
        printf("iteration: %d, chase_latency: %f \n",i, ((float)h_clock[i])/((float)chase_iter));
    }
  
}


iteration: 0, chase_latency: 16.395000 
iteration: 1, chase_latency: 16.389999 
iteration: 2, chase_latency: 16.389999 
iteration: 3, chase_latency: 16.389999 
iteration: 4, chase_latency: 16.389999 
iteration: 5, chase_latency: 16.389999 
iteration: 6, chase_latency: 16.389999 
iteration: 7, chase_latency: 16.389999 
iteration: 8, chase_latency: 16.389999 
iteration: 9, chase_latency: 16.389999 
iteration: 10, chase_latency: 16.389999 
iteration: 11, chase_latency: 16.389999 
iteration: 12, chase_latency: 16.389999 
iteration: 13, chase_latency: 16.389999 
iteration: 14, chase_latency: 16.389999 
iteration: 15, chase_latency: 16.389999 
iteration: 16, chase_latency: 16.389999 
iteration: 17, chase_latency: 16.389999 
iteration: 18, chase_latency: 16.389999 
iteration: 19, chase_latency: 16.389999 
iteration: 20, chase_latency: 16.389999 
iteration: 21, chase_latency: 16.389999 
iteration: 22, chase_latency: 16.389999 
iteration: 23, chase_latency: 16.389999 
iteration: 24, chase_laten

In [ ]:
%%cuda
#include <stdio.h> 
#include <stdlib.h>
#include <cuda_runtime.h> 
#include <device_launch_parameters.h>

#define CUDA_CHECK(call)                                                          \
    do {                                                                          \
        cudaError_t err = call;                                                   \
        if (err != cudaSuccess) {                                                 \
            fprintf(stderr, "CUDA error at %s:%d: %s\n",                          \
                    __FILE__, __LINE__, cudaGetErrorString(err));                 \
            exit(EXIT_FAILURE);                                                   \
        }                                                                         \
    } while (0)

constexpr int N_ITER_SAMPLES = 2000;  // Number of times to measure the chase sequence
constexpr int N_CHASE_LENGTH = 64;    // The length of the dependent load chain (N_CHASE)
constexpr int N_THREADS = 1;          // Ensure single thread execution
constexpr int N_BLOCKS = 1;

// 64-bit clock counter: This is correct for PTX `clock64`.
__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}

// Global kernel function
__global__ void smem_chase (float* chase_sequence, unsigned long long * clocks)
{
    // threadIdx.x is 0 since N_THREADS=1
    int t = threadIdx.x; 
    
    // Shared memory size is correctly set to N_CHASE_LENGTH * sizeof(float)
    __shared__ float S[N_CHASE_LENGTH];
    
    // --- 1. Load Chase Sequence from Global Memory (chase_sequence) to Shared Memory (S) ---
    // Since only one thread is running, it does the entire copy.
    for (int j = 0; j < N_CHASE_LENGTH; j++)
    {
        S[j] = chase_sequence[j];
    }
    
    // Not strictly necessary for one thread, but good practice before timing.
    __syncthreads();
    
    // --- 2. Pointer Chase Benchmark Loop ---
    for (int k = 0; k < N_ITER_SAMPLES; k++)
    {
        int idx = 0; // The index into S[] (always starts at S[0])
        void* ptr_addr; // Pointer for PTX address constraint
        unsigned long long s, e; 
        float r1; 
        
        s = get_clock64();
        
        // Dependent Chase Sequence
        for (int q = 0; q < N_CHASE_LENGTH; q++)
        {
            // 1. Get the correct shared memory address (NO casting to int before pointer)
            ptr_addr = (void*)&S[idx]; 
            
            // 2. Load and Store (DEPENDENCY ENFORCED)
            // Use 'l' constraint for 64-bit pointer address (more reliable than 'r')
            asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "l"(ptr_addr));
                    
            // 3. Update index using the loaded value to enforce pipeline stall
            idx = (int)r1;
            
            // Optional: Use the result to prevent the compiler from optimizing the whole chase away
            // (The dependency usually prevents this, but a safety measure)
            // S[0] = S[0] + r1; // Or some other non-critical write
        }
        e = get_clock64();
        
        // Store result: Since t=0, this stores at clock[k]
        clocks[k] = e - s;
    }
}


int main()
{
    // Corrected constant names and sizes
    constexpr int N_SAMPLES = N_ITER_SAMPLES;
    constexpr int N_CHASE = N_CHASE_LENGTH;

    size_t arr_size = N_CHASE * sizeof(float);
    size_t clock_arr_size = N_SAMPLES * sizeof(unsigned long long);
    
    float* h_chase, *d_chase;
    unsigned long long * h_clock, *d_clock;
    
    // --- 1. Allocation ---
    CUDA_CHECK(cudaHostAlloc(&h_chase, arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaHostAlloc(&h_clock, clock_arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaMalloc(&d_chase, arr_size));
    CUDA_CHECK(cudaMalloc(&d_clock, clock_arr_size));
    
    // --- 2. Host-Side Pointer Chase Sequence Generation ---
    // NOTE: The loop should run N_CHASE times to fill the array, not chase_iter times (which was 200)
    int index = 0;
    
    // The sequence is N_CHASE long.
    for (int q = 0; q < N_CHASE - 1; q++)
    {
        // Generates a random index between 0 and N_CHASE - 1
        h_chase[index] = (float) (rand() % N_CHASE); 
        index = (int) h_chase[index];
    }
    // Final element of the chase sequence MUST point back to the start (index 0) 
    // to form a loop/ring and ensure the chase never goes out of bounds.
    h_chase[index] = 0.0f;
    
    // --- 3. Execution ---
    CUDA_CHECK(cudaMemcpy(d_chase, h_chase, arr_size, cudaMemcpyHostToDevice));
    
    // Launch with 1 block, 1 thread
    smem_chase<<<N_BLOCKS, N_THREADS>>>(d_chase, d_clock); 
    
    // Wait for kernel completion and check for errors
    CUDA_CHECK(cudaDeviceSynchronize());
    
    // --- 4. Result Processing ---
    CUDA_CHECK(cudaMemcpy(h_clock, d_clock, clock_arr_size, cudaMemcpyDeviceToHost));
    
    float total_latency = 0.0f;
    for (int i = 0; i < N_SAMPLES; i++)
    {
        total_latency += (float)h_clock[i];
    }
    
    float avg_total_latency = total_latency / N_SAMPLES;
    float avg_single_load_latency = avg_total_latency / N_CHASE;

    printf("\n--- Shared Memory Pointer Chase Results ---\n");
    printf("Total Samples Measured: %d\n", N_SAMPLES);
    printf("Chase Chain Length (N): %d\n", N_CHASE);
    printf("Average Total Latency (T_N) per chase: %.2f cycles\n", avg_total_latency);
    printf("Estimated Single Load Latency (T_N / N): %.2f cycles\n", avg_single_load_latency);
    printf("-----------------------------------------\n");

    // --- 5. Cleanup ---
    cudaFree(d_chase);
    cudaFree(d_clock);
    cudaFreeHost(h_chase);
    cudaFreeHost(h_clock);

    return 0;
}


--- Shared Memory Pointer Chase Results ---
Total Samples Measured: 2000
Chase Chain Length (N): 64
Average Total Latency (T_N) per chase: 3278.02 cycles
Estimated Single Load Latency (T_N / N): 51.22 cycles
-----------------------------------------



Even that is not enough, cause still if we have perm[i] = j  and perm[j] = i, our chase sequence will loop i->j->i over and over, 

In [14]:
%%cuda
#include <stdio.h> 
#include <stdlib.h>
#include <cuda_runtime.h> 
#include <device_launch_parameters.h>

#define CUDA_CHECK(call)                                                          \
    do {                                                                          \
        cudaError_t err = call;                                                   \
        if (err != cudaSuccess) {                                                 \
            fprintf(stderr, "CUDA error at %s:%d: %s\n",                          \
                    __FILE__, __LINE__, cudaGetErrorString(err));                 \
            exit(EXIT_FAILURE);                                                   \
        }                                                                         \
    } while (0)

constexpr int N_ITER_SAMPLES = 10;  // Number of times to measure the chase sequence
constexpr int N_CHASE_LENGTH = 32;    // The length of the dependent load chain (N_CHASE)
constexpr int N_THREADS = 1;          // Ensure single thread execution
constexpr int N_BLOCKS = 1;

// 64-bit clock counter: This is correct for PTX `clock64`.
__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}

// Global kernel function
__global__ void smem_chase (float* chase_sequence, unsigned long long * clocks)
{
    // threadIdx.x is 0 since N_THREADS=1
    int t = threadIdx.x; 
    
    // Shared memory size is correctly set to N_CHASE_LENGTH * sizeof(float)
    __shared__ float S[N_CHASE_LENGTH];
    
    // --- 1. Load Chase Sequence from Global Memory (chase_sequence) to Shared Memory (S) ---
    // Since only one thread is running, it does the entire copy.
    for (int j = 0; j < N_CHASE_LENGTH; j++)
    {
        S[j] = chase_sequence[j];
    }
    
    // Not strictly necessary for one thread, but good practice before timing.
    __syncthreads();
    
    // --- 2. Pointer Chase Benchmark Loop ---
    for (int k = 0; k < N_ITER_SAMPLES; k++)
    {
        int idx = (int) S[0]; 
        void* ptr_addr; // Pointer for PTX address constraint
        unsigned long long s, e; 
        float r1; 
        
        s = get_clock64();
        
        // Dependent Chase Sequence
        for (int q = 0; q < N_CHASE_LENGTH; q++)
        {
            // 1. Get the correct shared memory address (NO casting to int before pointer)
            ptr_addr = (void*)&S[idx]; 
            
            // 2. Load and Store (DEPENDENCY ENFORCED)
            // Use 'l' constraint for 64-bit pointer address (more reliable than 'r')
            asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "l"(ptr_addr));
                    
            // 3. Update index using the loaded value to enforce pipeline stall
            printf("idx : %d, ", idx);
            idx = (int)r1;
            printf("next_idx : %d, \n", idx);
            
            
        }
        e = get_clock64();
        
        // Store result: Since t=0, this stores at clock[k]
        clocks[k] = e - s;
    }
}

void fisher_yates_shuffle(int *arr, int n) {
    if (n > 1) {
        for (int i = 0; i < n - 1; i++) {
            // Generate a random number between i and n-1
            int j = i + (rand() % (n - i));
            
            // Swap arr[i] and arr[j]
            int temp = arr[i];
            arr[i] = arr[j];
            arr[j] = temp;
        }
    }
}

int main()
{
    // Corrected constant names and sizes
    constexpr int N_SAMPLES = N_ITER_SAMPLES;
    constexpr int N_CHASE = N_CHASE_LENGTH;

    size_t arr_size = N_CHASE * sizeof(float);
    size_t clock_arr_size = N_SAMPLES * sizeof(unsigned long long);
    size_t indices_size = N_CHASE * sizeof(int);
    
    float* h_chase, *d_chase;
    unsigned long long * h_clock, *d_clock;
    
  
    CUDA_CHECK(cudaHostAlloc(&h_chase, arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaHostAlloc(&h_clock, clock_arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaMalloc(&d_chase, arr_size));
    CUDA_CHECK(cudaMalloc(&d_clock, clock_arr_size));
    
   
    int* indices = (int*) malloc(indices_size);
    for (int j = 0; j < N_CHASE; j++)
    {
        indices[j] = j;
    }
    
    fisher_yates_shuffle(indices, N_CHASE);
    for (int i = 0; i < N_CHASE - 1; i++) {
        // h_chase[Current Index] = Next Index (as a float)
        // Ensure h_chase is float* and indices is int*
        h_chase[indices[i]] = (float)indices[i+1]; 
    }

    // CLOSE THE LOOP: The last element in the sequence links back to the first.
    h_chase[indices[N_CHASE - 1]] = (float)indices[0];
    
    printf("==== lets print the chase sequence ===== \n");
    int idx = (int) h_chase[0]; 
    for (int j = 0; j < N_CHASE; j++)
    {
        int next_index = (int) h_chase[idx];
        
        printf("index: %d, next_index :%d \n", idx, next_index); 
        idx = next_index; 
    }
    printf("==== chase sequence over ===== \n");
    
    // --- 3. Execution ---
    CUDA_CHECK(cudaMemcpy(d_chase, h_chase, arr_size, cudaMemcpyHostToDevice));
    
    // Launch with 1 block, 1 thread
    smem_chase<<<N_BLOCKS, N_THREADS>>>(d_chase, d_clock); 
    
    // Wait for kernel completion and check for errors
    CUDA_CHECK(cudaDeviceSynchronize());
    
    // --- 4. Result Processing ---
    CUDA_CHECK(cudaMemcpy(h_clock, d_clock, clock_arr_size, cudaMemcpyDeviceToHost));
    
    float total_latency = 0.0f;
    for (int i = 0; i < N_SAMPLES; i++)
    {
        total_latency += (float)h_clock[i];
    }
    
    float avg_total_latency = total_latency / N_SAMPLES;
    float avg_single_load_latency = avg_total_latency / N_CHASE;

    printf("\n--- Shared Memory Pointer Chase Results ---\n");
    printf("Total Samples Measured: %d\n", N_SAMPLES);
    printf("Chase Chain Length (N): %d\n", N_CHASE);
    printf("Average Total Latency (T_N) per chase: %.2f cycles\n", avg_total_latency);
    printf("Estimated Single Load Latency (T_N / N): %.2f cycles\n", avg_single_load_latency);
    printf("-----------------------------------------\n");

    // --- 5. Cleanup ---
    cudaFree(d_chase);
    cudaFree(d_clock);
    cudaFreeHost(h_chase);
    cudaFreeHost(h_clock);

    return 0;
}

==== lets print the chase sequence ===== 
index: 16, next_index :25 
index: 25, next_index :9 
index: 9, next_index :27 
index: 27, next_index :26 
index: 26, next_index :8 
index: 8, next_index :7 
index: 7, next_index :6 
index: 6, next_index :29 
index: 29, next_index :28 
index: 28, next_index :5 
index: 5, next_index :30 
index: 30, next_index :18 
index: 18, next_index :24 
index: 24, next_index :17 
index: 17, next_index :10 
index: 10, next_index :1 
index: 1, next_index :4 
index: 4, next_index :22 
index: 22, next_index :23 
index: 23, next_index :19 
index: 19, next_index :20 
index: 20, next_index :3 
index: 3, next_index :13 
index: 13, next_index :15 
index: 15, next_index :11 
index: 11, next_index :31 
index: 31, next_index :2 
index: 2, next_index :21 
index: 21, next_index :12 
index: 12, next_index :14 
index: 14, next_index :0 
index: 0, next_index :16 
==== chase sequence over ===== 
idx : 16, next_idx : 25, 
idx : 25, next_idx : 9, 
idx : 9, next_idx : 27, 
idx : 

Okay, the above is not the benchmark of course, we gotta clean it up, and moreover, we'd need to remove out the time for actually doing the arithmetic to find the next index. Lets do that!: 

In [16]:
%%cuda
#include <stdio.h> 
#include <stdlib.h>
#include <cuda_runtime.h> 
#include <device_launch_parameters.h>

#define CUDA_CHECK(call)                                                          \
    do {                                                                          \
        cudaError_t err = call;                                                   \
        if (err != cudaSuccess) {                                                 \
            fprintf(stderr, "CUDA error at %s:%d: %s\n",                          \
                    __FILE__, __LINE__, cudaGetErrorString(err));                 \
            exit(EXIT_FAILURE);                                                   \
        }                                                                         \
    } while (0)

constexpr int N_ITER_SAMPLES = 100;  
constexpr int N_CHASE_LENGTH = 128;    
constexpr int N_THREADS = 1;          
constexpr int N_BLOCKS = 1;


__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}


__global__ void smem_chase (float* chase_sequence, unsigned long long * clocks)
{

    int t = threadIdx.x; 

    __shared__ float S[N_CHASE_LENGTH];
    
    for (int j = 0; j < N_CHASE_LENGTH; j++)
    {
        S[j] = chase_sequence[j];
    }
    
    __syncthreads();

    for (int k = 0; k < N_ITER_SAMPLES; k++)
    {
        int idx = (int) S[0]; 
        void* ptr_addr; 
        unsigned long long s, e; 
        float r1; 
        
        s = get_clock64();

        for (int q = 0; q < N_CHASE_LENGTH; q++)
        {
            ptr_addr = (void*)&S[idx]; 
            
            asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "l"(ptr_addr));
                    
            idx = (int)r1;

        }
        
        e = get_clock64();
        
        clocks[k] = e - s;
    }
}

void fisher_yates_shuffle(int *arr, int n) {
    if (n > 1) {
        for (int i = 0; i < n - 1; i++) {
            // Generate a random number between i and n-1
            int j = i + (rand() % (n - i));
            
            // Swap arr[i] and arr[j]
            int temp = arr[i];
            arr[i] = arr[j];
            arr[j] = temp;
        }
    }
}

int main()
{
    // Corrected constant names and sizes
    constexpr int N_SAMPLES = N_ITER_SAMPLES;
    constexpr int N_CHASE = N_CHASE_LENGTH;

    size_t arr_size = N_CHASE * sizeof(float);
    size_t clock_arr_size = N_SAMPLES * sizeof(unsigned long long);
    size_t indices_size = N_CHASE * sizeof(int);
    
    float* h_chase, *d_chase;
    unsigned long long * h_clock, *d_clock;
    
  
    CUDA_CHECK(cudaHostAlloc(&h_chase, arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaHostAlloc(&h_clock, clock_arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaMalloc(&d_chase, arr_size));
    CUDA_CHECK(cudaMalloc(&d_clock, clock_arr_size));
    
   
    int* indices = (int*) malloc(indices_size);
    for (int j = 0; j < N_CHASE; j++)
    {
        indices[j] = j;
    }
    
    fisher_yates_shuffle(indices, N_CHASE);
    for (int i = 0; i < N_CHASE - 1; i++) {

        h_chase[indices[i]] = (float)indices[i+1]; 
    }

    h_chase[indices[N_CHASE - 1]] = (float)indices[0];
    
    /*
    """
    printf("==== lets print the chase sequence ===== \n");
    int idx = (int) h_chase[0]; 
    for (int j = 0; j < N_CHASE; j++)
    {
        int next_index = (int) h_chase[idx];
        
        printf("index: %d, next_index :%d \n", idx, next_index); 
        idx = next_index; 
        
    }
    printf("==== chase sequence over ===== \n");
    """
    */
    
    // --- 3. Execution ---
    CUDA_CHECK(cudaMemcpy(d_chase, h_chase, arr_size, cudaMemcpyHostToDevice));
    
    // Launch with 1 block, 1 thread
    smem_chase<<<N_BLOCKS, N_THREADS>>>(d_chase, d_clock); 
    
    // Wait for kernel completion and check for errors
    CUDA_CHECK(cudaDeviceSynchronize());
    
    // --- 4. Result Processing ---
    CUDA_CHECK(cudaMemcpy(h_clock, d_clock, clock_arr_size, cudaMemcpyDeviceToHost));
    
    float total_latency = 0.0f;
    for (int i = 0; i < N_SAMPLES; i++)
    {
        total_latency += (float)h_clock[i];
    }
    
    float avg_total_latency = total_latency / N_SAMPLES;
    float avg_single_load_latency = avg_total_latency / N_CHASE;

    printf("\n--- Shared Memory Pointer Chase Results ---\n");
    printf("Total Samples Measured: %d\n", N_SAMPLES);
    printf("Chase Chain Length (N): %d\n", N_CHASE);
    printf("Average Total Latency (T_N) per chase: %.2f cycles\n", avg_total_latency);
    printf("Estimated Single Load Latency (T_N / N): %.2f cycles\n", avg_single_load_latency);
    printf("-----------------------------------------\n");

    // --- 5. Cleanup ---
    cudaFree(d_chase);
    cudaFree(d_clock);
    cudaFreeHost(h_chase);
    cudaFreeHost(h_clock);

    return 0;
}


--- Shared Memory Pointer Chase Results ---
Total Samples Measured: 100
Chase Chain Length (N): 128
Average Total Latency (T_N) per chase: 6633.00 cycles
Estimated Single Load Latency (T_N / N): 51.82 cycles
-----------------------------------------



okay but now, we want to cut out the arithmetic bits that are you know, contributing to this. One Thing 
we can try is to simply fucking delete the asm, but since the arithmetic bits are in cuda, they might be thrown out of the compiler. lets see :) :) 

In [17]:
%%cuda
#include <stdio.h> 
#include <stdlib.h>
#include <cuda_runtime.h> 
#include <device_launch_parameters.h>

#define CUDA_CHECK(call)                                                          \
    do {                                                                          \
        cudaError_t err = call;                                                   \
        if (err != cudaSuccess) {                                                 \
            fprintf(stderr, "CUDA error at %s:%d: %s\n",                          \
                    __FILE__, __LINE__, cudaGetErrorString(err));                 \
            exit(EXIT_FAILURE);                                                   \
        }                                                                         \
    } while (0)

constexpr int N_ITER_SAMPLES = 100;  
constexpr int N_CHASE_LENGTH = 128;    
constexpr int N_THREADS = 1;          
constexpr int N_BLOCKS = 1;


__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}


__global__ void smem_chase (float* chase_sequence, unsigned long long * clocks)
{

    int t = threadIdx.x; 

    __shared__ float S[N_CHASE_LENGTH];
    
    for (int j = 0; j < N_CHASE_LENGTH; j++)
    {
        S[j] = chase_sequence[j];
    }
    
    __syncthreads();

    for (int k = 0; k < N_ITER_SAMPLES; k++)
    {
        int idx = (int) S[0]; 
        void* ptr_addr; 
        unsigned long long s, e; 
        float r1; 
        
        s = get_clock64();

        for (int q = 0; q < N_CHASE_LENGTH; q++)
        {
            ptr_addr = (void*)&S[idx]; 
            

            idx = (int)r1;

        }
        
        e = get_clock64();
        
        clocks[k] = e - s;
    }
}

void fisher_yates_shuffle(int *arr, int n) {
    if (n > 1) {
        for (int i = 0; i < n - 1; i++) {
            // Generate a random number between i and n-1
            int j = i + (rand() % (n - i));
            
            // Swap arr[i] and arr[j]
            int temp = arr[i];
            arr[i] = arr[j];
            arr[j] = temp;
        }
    }
}

int main()
{
    // Corrected constant names and sizes
    constexpr int N_SAMPLES = N_ITER_SAMPLES;
    constexpr int N_CHASE = N_CHASE_LENGTH;

    size_t arr_size = N_CHASE * sizeof(float);
    size_t clock_arr_size = N_SAMPLES * sizeof(unsigned long long);
    size_t indices_size = N_CHASE * sizeof(int);
    
    float* h_chase, *d_chase;
    unsigned long long * h_clock, *d_clock;
    
  
    CUDA_CHECK(cudaHostAlloc(&h_chase, arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaHostAlloc(&h_clock, clock_arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaMalloc(&d_chase, arr_size));
    CUDA_CHECK(cudaMalloc(&d_clock, clock_arr_size));
    
   
    int* indices = (int*) malloc(indices_size);
    for (int j = 0; j < N_CHASE; j++)
    {
        indices[j] = j;
    }
    
    fisher_yates_shuffle(indices, N_CHASE);
    for (int i = 0; i < N_CHASE - 1; i++) {

        h_chase[indices[i]] = (float)indices[i+1]; 
    }

    h_chase[indices[N_CHASE - 1]] = (float)indices[0];
    
    /*
    """
    printf("==== lets print the chase sequence ===== \n");
    int idx = (int) h_chase[0]; 
    for (int j = 0; j < N_CHASE; j++)
    {
        int next_index = (int) h_chase[idx];
        
        printf("index: %d, next_index :%d \n", idx, next_index); 
        idx = next_index; 
        
    }
    printf("==== chase sequence over ===== \n");
    """
    */
    
    // --- 3. Execution ---
    CUDA_CHECK(cudaMemcpy(d_chase, h_chase, arr_size, cudaMemcpyHostToDevice));
    
    // Launch with 1 block, 1 thread
    smem_chase<<<N_BLOCKS, N_THREADS>>>(d_chase, d_clock); 
    
    // Wait for kernel completion and check for errors
    CUDA_CHECK(cudaDeviceSynchronize());
    
    // --- 4. Result Processing ---
    CUDA_CHECK(cudaMemcpy(h_clock, d_clock, clock_arr_size, cudaMemcpyDeviceToHost));
    
    float total_latency = 0.0f;
    for (int i = 0; i < N_SAMPLES; i++)
    {
        total_latency += (float)h_clock[i];
    }
    
    float avg_total_latency = total_latency / N_SAMPLES;
    float avg_single_load_latency = avg_total_latency / N_CHASE;

    printf("\n--- Shared Memory Pointer Chase Results ---\n");
    printf("Total Samples Measured: %d\n", N_SAMPLES);
    printf("Chase Chain Length (N): %d\n", N_CHASE);
    printf("Average Total Latency (T_N) per chase: %.2f cycles\n", avg_total_latency);
    printf("Estimated Single Load Latency (T_N / N): %.2f cycles\n", avg_single_load_latency);
    printf("-----------------------------------------\n");

    // --- 5. Cleanup ---
    cudaFree(d_chase);
    cudaFree(d_clock);
    cudaFreeHost(h_chase);
    cudaFreeHost(h_clock);

    return 0;
}


--- Shared Memory Pointer Chase Results ---
Total Samples Measured: 100
Chase Chain Length (N): 128
Average Total Latency (T_N) per chase: 14.86 cycles
Estimated Single Load Latency (T_N / N): 0.12 cycles
-----------------------------------------



yeah and the logic of it doesn't make sense either. we need to create a register array of the shared memory and then compute the chase sequence from there. 

In [22]:
%%cuda
#include <stdio.h> 
#include <stdlib.h>
#include <cuda_runtime.h> 
#include <device_launch_parameters.h>

#define CUDA_CHECK(call)                                                          \
    do {                                                                          \
        cudaError_t err = call;                                                   \
        if (err != cudaSuccess) {                                                 \
            fprintf(stderr, "CUDA error at %s:%d: %s\n",                          \
                    __FILE__, __LINE__, cudaGetErrorString(err));                 \
            exit(EXIT_FAILURE);                                                   \
        }                                                                         \
    } while (0)

constexpr int N_ITER_SAMPLES = 100;  
constexpr int N_CHASE_LENGTH = 32;    
constexpr int N_THREADS = 1;          
constexpr int N_BLOCKS = 1;


__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}


__global__ void smem_chase (float* chase_sequence, unsigned long long * clocks)
{

    int t = threadIdx.x; 

    float S_reg[N_CHASE_LENGTH];
    
    for (int j = 0; j < N_CHASE_LENGTH; j++)
    {
        S_reg[j] = chase_sequence[j];
    }
    
    __syncthreads();

    for (int k = 0; k < N_ITER_SAMPLES; k++)
    {
        int idx = (int) S_reg[0]; 
        unsigned long long s, e; 
        float r1; 
        
        s = get_clock64();

        for (int q = 0; q < N_CHASE_LENGTH; q++)
        {
            r1 = S_reg[idx];
            S_reg[idx] = r1; 
            idx = (int)r1;

        }
        
        e = get_clock64();
        
        int g = k % N_CHASE_LENGTH; 
        
        chase_sequence[g] = S_reg[g];
        clocks[k] = e - s;
        
    }
}

void fisher_yates_shuffle(int *arr, int n) {
    if (n > 1) {
        for (int i = 0; i < n - 1; i++) {
            // Generate a random number between i and n-1
            int j = i + (rand() % (n - i));
            
            // Swap arr[i] and arr[j]
            int temp = arr[i];
            arr[i] = arr[j];
            arr[j] = temp;
        }
    }
}

int main()
{
    // Corrected constant names and sizes
    constexpr int N_SAMPLES = N_ITER_SAMPLES;
    constexpr int N_CHASE = N_CHASE_LENGTH;

    size_t arr_size = N_CHASE * sizeof(float);
    size_t clock_arr_size = N_SAMPLES * sizeof(unsigned long long);
    size_t indices_size = N_CHASE * sizeof(int);
    
    float* h_chase, *d_chase;
    unsigned long long * h_clock, *d_clock;
    
  
    CUDA_CHECK(cudaHostAlloc(&h_chase, arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaHostAlloc(&h_clock, clock_arr_size, cudaHostAllocDefault));
    CUDA_CHECK(cudaMalloc(&d_chase, arr_size));
    CUDA_CHECK(cudaMalloc(&d_clock, clock_arr_size));
    
   
    int* indices = (int*) malloc(indices_size);
    for (int j = 0; j < N_CHASE; j++)
    {
        indices[j] = j;
    }
    
    fisher_yates_shuffle(indices, N_CHASE);
    for (int i = 0; i < N_CHASE - 1; i++) {

        h_chase[indices[i]] = (float)indices[i+1]; 
    }

    h_chase[indices[N_CHASE - 1]] = (float)indices[0];
    
    /*
    """
    printf("==== lets print the chase sequence ===== \n");
    int idx = (int) h_chase[0]; 
    for (int j = 0; j < N_CHASE; j++)
    {
        int next_index = (int) h_chase[idx];
        
        printf("index: %d, next_index :%d \n", idx, next_index); 
        idx = next_index; 
        
    }
    printf("==== chase sequence over ===== \n");
    """
    */
    
    // --- 3. Execution ---
    CUDA_CHECK(cudaMemcpy(d_chase, h_chase, arr_size, cudaMemcpyHostToDevice));
    
    // Launch with 1 block, 1 thread
    smem_chase<<<N_BLOCKS, N_THREADS>>>(d_chase, d_clock); 
    
    // Wait for kernel completion and check for errors
    CUDA_CHECK(cudaDeviceSynchronize());
    
    // --- 4. Result Processing ---
    CUDA_CHECK(cudaMemcpy(h_clock, d_clock, clock_arr_size, cudaMemcpyDeviceToHost));
    
    float total_latency = 0.0f;
    for (int i = 0; i < N_SAMPLES; i++)
    {
        total_latency += (float)h_clock[i];
    }
    
    float avg_total_latency = total_latency / N_SAMPLES;
    float avg_single_load_latency = avg_total_latency / N_CHASE;

    printf("\n--- Shared Memory Pointer Chase Results ---\n");
    printf("Total Samples Measured: %d\n", N_SAMPLES);
    printf("Chase Chain Length (N): %d\n", N_CHASE);
    printf("Average Total Latency (T_N) per chase: %.2f cycles\n", avg_total_latency);
    printf("Estimated Single Load Latency (T_N / N): %.2f cycles\n", avg_single_load_latency);
    printf("-----------------------------------------\n");

    // --- 5. Cleanup ---
    cudaFree(d_chase);
    cudaFree(d_clock);
    cudaFreeHost(h_chase);
    cudaFreeHost(h_clock);

    return 0;
}


--- Shared Memory Pointer Chase Results ---
Total Samples Measured: 100
Chase Chain Length (N): 32
Average Total Latency (T_N) per chase: 16.01 cycles
Estimated Single Load Latency (T_N / N): 0.50 cycles
-----------------------------------------



Okay whatever, essentially, the SMEM latency is about 30-50 clock cycles, as said by the ![[https://arxiv.org/pdf/2507.10789]] Dissecting nvidia blackwell architecture. lets us move on now, and use clock cycles to measure SMEM throughput and stuff like that. 

What we want is to go back to the kernel with $N$ warps and a really nice access pattern. 